In [1]:
import import_ipynb
import tensorflow as tf
from fin_data import data_generator
from keras.models import Sequential
from keras.layers.merge import Add
from keras.optimizers import Adam,Nadam
from models import discriminator_model, generator_model_cnn,generator_model_mlp,generator_model_mlp_cnn,generator_model_mlp_cnn_plus
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
#import visualize
#import stylized_facts as sf
from datetime import datetime as dt
import os
#from shutil import copyfile
#import argparse 

# parser = argparse.ArgumentParser(description='FIN-GAN implementation')
# parser.add_argument('--batch_size',type=int,default=24)
# parser.add_argument('--generator_model',type=str,default='mlp-cnn')
# parser.add_argument('--epochs',type=int,default=10)
# parser.add_argument('--batches',type=int,default=1024)
# parser.add_argument('--folder_name',type=str,default='')
# parser.add_argument('--generator_lr',type=float,default='2e-4')
# parser.add_argument('--discriminator_lr',type=float,default='1e-5')
# parser.add_argument('--log_interval',type=int,default=50)
# parser.add_argument('--seed',type=int,default=1)

#args = parser.parse_args()
#seed(args.seed)
#set_random_seed(args.seed)


importing Jupyter notebook from fin_data.ipynb
importing Jupyter notebook from data.ipynb


Using TensorFlow backend.


importing Jupyter notebook from models.ipynb


In [2]:
dg = data_generator()
batch_size = 24
dg.batch_size = batch_size
batches = 1024
epochs = 10                 
timestamp = dt.now().strftime("%Y%m%d_%H%M%S")
timestamp += '_'
#timestamp += args.folder_name
# os.mkdir('./imgs/%s'%(timestamp))
# os.mkdir('./npy/%s'%(timestamp))
# os.mkdir('./weights/%s'%(timestamp))
# os.mkdir('./imgs/%s/acf'%(timestamp))
# os.mkdir('./imgs/%s/dist'%(timestamp))
# os.mkdir('./imgs/%s/time_series'%(timestamp))
# os.mkdir('./imgs/%s/leverage'%(timestamp))
# copyfile('./main.py','./imgs/%s/main.py'%(timestamp))
# copyfile('./models.py','./imgs/%s/models.py'%(timestamp))
# with open('./imgs/%s/hyper_parameters.txt'%(timestamp),'w') as w:
#     w.write(str(args))
def train():
    generator,generator_statistics = generator_model_mlp_cnn()
    #preparing generator
#     if args.generator_model == 'mlp-cnn':
#         generator,generator_statistics = generator_model_mlp_cnn()
#     elif args.generator_model == 'mlp':
#         generator,generator_statistics = generator_model_mlp()
#     elif args.generator_model == 'cnn':
#         generator = generator_model_cnn()
#     elif args.generator_model == 'plus':
#         generator = generator_model_mlp_cnn_plus()
#     else:
#         import sys
#         sys.exit()
    #preparing discriminator
                                                      
    statistics_opt = Adam(lr=0.0001)
    generator_statistics.compile(loss='mean_squared_error',optimizer=statistics_opt)

    discriminator = discriminator_model()
    d_opt = Adam(lr=1e-5, beta_1=0.1)
    discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)
    discriminator.trainable = False
    for e in discriminator.layers:
        e.trainable = False
    gan = Sequential([generator,discriminator])
    g_opt = Adam(lr=2e-4, beta_1=0.5)
    gan.compile(loss='binary_crossentropy', optimizer=g_opt)
    
    g_loss_recorder = []
    d_loss_recorder = []
    g_losses_recorder = []
    d_losses_recorder = []
    #start training
    for epoch in range(epochs):
        for index in range(batches):
            noise = np.array([np.random.normal(0,1.0,size=100) for _ in range(batch_size)])
            real_series = dg.index_data_random_picker()
            real_series = np.nan_to_num(real_series)
            generated_series = generator.predict(noise, verbose=0)
#             if index % args.log_interval == 0:
#                 sf.acf(generated_series,'./imgs/%s/acf/acf_abs_%i_%i'%(timestamp,epoch,index),for_abs=True)
#                 sf.acf(generated_series,'./imgs/%s/acf/acf_raw_%i_%i'%(timestamp,epoch,index),for_abs=False)
#                 sf.acf(generated_series,'./imgs/%s/acf/acf_abs_linear_%i_%i'%(timestamp,epoch,index),for_abs=True,scale='linear')
#                 sf.acf(generated_series,'./imgs/%s/acf/acf_raw_linear_%i_%i'%(timestamp,epoch,index),for_abs=False,scale='linear')
#                 sf.leverage_effect(generated_series,'./imgs/%s/leverage/leverage_%i_%i'%(timestamp,epoch,index))
#                 sf.distribution(generated_series, './imgs/%s/dist/distribution_%i_%i'%(timestamp,epoch,index),'linear')
#                 sf.distribution(generated_series, './imgs/%s/dist/distribution_%i_%i'%(timestamp,epoch,index),'log')
#                 visualize.time_series(generated_series[0],'./imgs/%s/time_series/generated_time_series_%i_%i'%(timestamp,epoch,index))
#                 np.save('./npy/%s/generated_time_series_%i_%i.npy'%(timestamp,epoch,index),generated_series)
            # update discriminator
            X = np.concatenate((real_series, generated_series))
            y = np.concatenate([np.random.uniform(0.9,1.1,batch_size),np.random.uniform(0.1,0.3,batch_size)])
            d_loss = discriminator.train_on_batch(X, y)
            d_loss_recorder.append(d_loss)
            # update generator
            y = np.array([1.]*batch_size,dtype=np.float32)
            g_loss = gan.train_on_batch(noise, y)
            g_loss_recorder.append(g_loss)
            print("epoch: %d, batch: %d, g_loss: %f, d_loss: %f" % (epoch, index, g_loss, d_loss))
            #generator.save_weights('./weights/%s/generator_%i_%i.h5'%(timestamp,epoch,index))

In [ ]:
train()

epoch: 0, batch: 0, g_loss: 0.657158, d_loss: 0.692976
epoch: 0, batch: 1, g_loss: 0.613787, d_loss: 0.692266
epoch: 0, batch: 2, g_loss: 0.571954, d_loss: 0.692044
epoch: 0, batch: 3, g_loss: 0.556436, d_loss: 0.692596
epoch: 0, batch: 4, g_loss: 0.611370, d_loss: 0.696136
epoch: 0, batch: 5, g_loss: 0.676628, d_loss: 0.695459
epoch: 0, batch: 6, g_loss: 0.781272, d_loss: 0.694235
epoch: 0, batch: 7, g_loss: 0.823267, d_loss: 0.688857
epoch: 0, batch: 8, g_loss: 0.855785, d_loss: 0.688779
epoch: 0, batch: 9, g_loss: 0.863140, d_loss: 0.687528
epoch: 0, batch: 10, g_loss: 0.900495, d_loss: 0.689059
epoch: 0, batch: 11, g_loss: 0.878974, d_loss: 0.692836
epoch: 0, batch: 12, g_loss: 0.828680, d_loss: 0.693946
epoch: 0, batch: 13, g_loss: 0.792214, d_loss: 0.693763
epoch: 0, batch: 14, g_loss: 0.778899, d_loss: 0.692997
epoch: 0, batch: 15, g_loss: 0.777317, d_loss: 0.692117
epoch: 0, batch: 16, g_loss: 0.770794, d_loss: 0.691019
epoch: 0, batch: 17, g_loss: 0.787408, d_loss: 0.689429
ep

epoch: 0, batch: 146, g_loss: 0.497829, d_loss: 0.669114
epoch: 0, batch: 147, g_loss: 0.495170, d_loss: 0.671418
epoch: 0, batch: 148, g_loss: 0.508089, d_loss: 0.676083
epoch: 0, batch: 149, g_loss: 0.540384, d_loss: 0.680209
epoch: 0, batch: 150, g_loss: 0.534901, d_loss: 0.674488
epoch: 0, batch: 151, g_loss: 0.584089, d_loss: 0.679826
epoch: 0, batch: 152, g_loss: 0.531981, d_loss: 0.660456
epoch: 0, batch: 153, g_loss: 0.546543, d_loss: 0.679739
epoch: 0, batch: 154, g_loss: 0.551891, d_loss: 0.677887
epoch: 0, batch: 155, g_loss: 0.537364, d_loss: 0.668547
epoch: 0, batch: 156, g_loss: 0.600134, d_loss: 0.672996
epoch: 0, batch: 157, g_loss: 0.584188, d_loss: 0.665795
epoch: 0, batch: 158, g_loss: 0.576975, d_loss: 0.694988
epoch: 0, batch: 159, g_loss: 0.589222, d_loss: 0.664781
epoch: 0, batch: 160, g_loss: 0.558298, d_loss: 0.666114
epoch: 0, batch: 161, g_loss: 0.568410, d_loss: 0.675210
epoch: 0, batch: 162, g_loss: 0.581676, d_loss: 0.665343
epoch: 0, batch: 163, g_loss: 0

epoch: 0, batch: 290, g_loss: 0.545247, d_loss: 0.654255
epoch: 0, batch: 291, g_loss: 0.560125, d_loss: 0.667079
epoch: 0, batch: 292, g_loss: 0.529846, d_loss: 0.655637
epoch: 0, batch: 293, g_loss: 0.511895, d_loss: 0.660922
epoch: 0, batch: 294, g_loss: 0.574677, d_loss: 0.672740
epoch: 0, batch: 295, g_loss: 0.548967, d_loss: 0.656815
epoch: 0, batch: 296, g_loss: 0.567726, d_loss: 0.670683
epoch: 0, batch: 297, g_loss: 0.550194, d_loss: 0.657509
epoch: 0, batch: 298, g_loss: 0.532107, d_loss: 0.652547
epoch: 0, batch: 299, g_loss: 0.528419, d_loss: 0.669332
epoch: 0, batch: 300, g_loss: 0.528218, d_loss: 0.653155
epoch: 0, batch: 301, g_loss: 0.573953, d_loss: 0.657631
epoch: 0, batch: 302, g_loss: 0.520252, d_loss: 0.652283
epoch: 0, batch: 303, g_loss: 0.580152, d_loss: 0.661870
epoch: 0, batch: 304, g_loss: 0.526029, d_loss: 0.648385
epoch: 0, batch: 305, g_loss: 0.563493, d_loss: 0.663626
epoch: 0, batch: 306, g_loss: 0.520743, d_loss: 0.673793
epoch: 0, batch: 307, g_loss: 0

epoch: 0, batch: 434, g_loss: 0.523649, d_loss: 0.674924
epoch: 0, batch: 435, g_loss: 0.507813, d_loss: 0.654818
epoch: 0, batch: 436, g_loss: 0.510100, d_loss: 0.678610
epoch: 0, batch: 437, g_loss: 0.527013, d_loss: 0.669488
epoch: 0, batch: 438, g_loss: 0.521708, d_loss: 0.658887
epoch: 0, batch: 439, g_loss: 0.515909, d_loss: 0.667034
epoch: 0, batch: 440, g_loss: 0.542020, d_loss: 0.672385
epoch: 0, batch: 441, g_loss: 0.522461, d_loss: 0.675190
epoch: 0, batch: 442, g_loss: 0.550184, d_loss: 0.663479
epoch: 0, batch: 443, g_loss: 0.562591, d_loss: 0.682962
epoch: 0, batch: 444, g_loss: 0.545655, d_loss: 0.666780
epoch: 0, batch: 445, g_loss: 0.569984, d_loss: 0.667337
epoch: 0, batch: 446, g_loss: 0.542327, d_loss: 0.669173
epoch: 0, batch: 447, g_loss: 0.533923, d_loss: 0.656503
epoch: 0, batch: 448, g_loss: 0.535806, d_loss: 0.658975
epoch: 0, batch: 449, g_loss: 0.524632, d_loss: 0.661991
epoch: 0, batch: 450, g_loss: 0.517539, d_loss: 0.674103
epoch: 0, batch: 451, g_loss: 0

epoch: 0, batch: 578, g_loss: 0.556074, d_loss: 0.663444
epoch: 0, batch: 579, g_loss: 0.546258, d_loss: 0.665754
epoch: 0, batch: 580, g_loss: 0.517552, d_loss: 0.654589
epoch: 0, batch: 581, g_loss: 0.528593, d_loss: 0.665016
epoch: 0, batch: 582, g_loss: 0.538527, d_loss: 0.657051
epoch: 0, batch: 583, g_loss: 0.541090, d_loss: 0.678889
epoch: 0, batch: 584, g_loss: 0.540235, d_loss: 0.657663
epoch: 0, batch: 585, g_loss: 0.520170, d_loss: 0.665212
epoch: 0, batch: 586, g_loss: 0.531651, d_loss: 0.668209
epoch: 0, batch: 587, g_loss: 0.568469, d_loss: 0.673830
epoch: 0, batch: 588, g_loss: 0.537151, d_loss: 0.662058
epoch: 0, batch: 589, g_loss: 0.534568, d_loss: 0.650757
epoch: 0, batch: 590, g_loss: 0.532235, d_loss: 0.662961
epoch: 0, batch: 591, g_loss: 0.539326, d_loss: 0.665254
epoch: 0, batch: 592, g_loss: 0.556335, d_loss: 0.647981
epoch: 0, batch: 593, g_loss: 0.563651, d_loss: 0.675674
epoch: 0, batch: 594, g_loss: 0.528510, d_loss: 0.664530
epoch: 0, batch: 595, g_loss: 0

epoch: 0, batch: 722, g_loss: 0.542394, d_loss: 0.665751
epoch: 0, batch: 723, g_loss: 0.557294, d_loss: 0.660168
epoch: 0, batch: 724, g_loss: 0.543838, d_loss: 0.667515
epoch: 0, batch: 725, g_loss: 0.544190, d_loss: 0.669256
epoch: 0, batch: 726, g_loss: 0.554059, d_loss: 0.662164
epoch: 0, batch: 727, g_loss: 0.574896, d_loss: 0.667499
epoch: 0, batch: 728, g_loss: 0.551196, d_loss: 0.668840
epoch: 0, batch: 729, g_loss: 0.553701, d_loss: 0.658760
epoch: 0, batch: 730, g_loss: 0.559617, d_loss: 0.661047
epoch: 0, batch: 731, g_loss: 0.544783, d_loss: 0.663460
epoch: 0, batch: 732, g_loss: 0.557067, d_loss: 0.666106
epoch: 0, batch: 733, g_loss: 0.559720, d_loss: 0.658089
epoch: 0, batch: 734, g_loss: 0.531989, d_loss: 0.653130
epoch: 0, batch: 735, g_loss: 0.558637, d_loss: 0.653570
epoch: 0, batch: 736, g_loss: 0.541212, d_loss: 0.659408
epoch: 0, batch: 737, g_loss: 0.539198, d_loss: 0.652006
epoch: 0, batch: 738, g_loss: 0.508478, d_loss: 0.661615
epoch: 0, batch: 739, g_loss: 0

epoch: 0, batch: 866, g_loss: 0.591178, d_loss: 0.671518
epoch: 0, batch: 867, g_loss: 0.563299, d_loss: 0.640439
epoch: 0, batch: 868, g_loss: 0.544530, d_loss: 0.661855
epoch: 0, batch: 869, g_loss: 0.543217, d_loss: 0.641722
epoch: 0, batch: 870, g_loss: 0.527822, d_loss: 0.659513
epoch: 0, batch: 871, g_loss: 0.522187, d_loss: 0.651442
epoch: 0, batch: 872, g_loss: 0.561397, d_loss: 0.645318
epoch: 0, batch: 873, g_loss: 0.534371, d_loss: 0.671052
epoch: 0, batch: 874, g_loss: 0.539128, d_loss: 0.662502
epoch: 0, batch: 875, g_loss: 0.558692, d_loss: 0.669606
epoch: 0, batch: 876, g_loss: 0.524810, d_loss: 0.647281
epoch: 0, batch: 877, g_loss: 0.531114, d_loss: 0.653801
epoch: 0, batch: 878, g_loss: 0.568015, d_loss: 0.632506
epoch: 0, batch: 879, g_loss: 0.526276, d_loss: 0.653521
epoch: 0, batch: 880, g_loss: 0.527586, d_loss: 0.643287
epoch: 0, batch: 881, g_loss: 0.523021, d_loss: 0.642378
epoch: 0, batch: 882, g_loss: 0.533973, d_loss: 0.656605
epoch: 0, batch: 883, g_loss: 0

epoch: 0, batch: 1010, g_loss: 0.541061, d_loss: 0.665507
epoch: 0, batch: 1011, g_loss: 0.526853, d_loss: 0.651550
epoch: 0, batch: 1012, g_loss: 0.508074, d_loss: 0.644323
epoch: 0, batch: 1013, g_loss: 0.531094, d_loss: 0.658060
epoch: 0, batch: 1014, g_loss: 0.559381, d_loss: 0.661012
epoch: 0, batch: 1015, g_loss: 0.503031, d_loss: 0.640819
epoch: 0, batch: 1016, g_loss: 0.531049, d_loss: 0.651585
epoch: 0, batch: 1017, g_loss: 0.512043, d_loss: 0.655061
epoch: 0, batch: 1018, g_loss: 0.517899, d_loss: 0.655918
epoch: 0, batch: 1019, g_loss: 0.570373, d_loss: 0.668414
epoch: 0, batch: 1020, g_loss: 0.552247, d_loss: 0.657774
epoch: 0, batch: 1021, g_loss: 0.540935, d_loss: 0.652892
epoch: 0, batch: 1022, g_loss: 0.589381, d_loss: 0.656400
epoch: 0, batch: 1023, g_loss: 0.515479, d_loss: 0.655017
epoch: 1, batch: 0, g_loss: 0.572937, d_loss: 0.663658
epoch: 1, batch: 1, g_loss: 0.583272, d_loss: 0.670348
epoch: 1, batch: 2, g_loss: 0.550718, d_loss: 0.659899
epoch: 1, batch: 3, g_l